In [1]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
from cellpose import utils, io, transforms
import skimage
import cv2
from skimage.measure import label, regionprops
import re
import tifffile as tiff
from tqdm import tqdm
import scipy
import pandas as pd

In [2]:
def get_images_from_channels(image_file, image_paths_in):
    channel_files = ["w1", "w2", "w3", "w4", "w5"]
    images = {}
    for i in channel_files:
        file_name = image_file.split("w")[0]
        path_temp = os.path.join(folder_path, file_name) + i
        path_string = next((string for string in image_paths_in if path_temp in string), None)
        images[i] = skimage.io.imread(path_string)
    return images

def generate_single_cell_crops(mask, image_path):
    unique_ids = np.unique(mask.flatten())
    images = get_images_from_channels(image_path, image_paths_in = image_paths)
    for cell_id in unique_ids:
            if cell_id == 0:
                continue  # Skip the background
            single_cell_mask = np.where(mask == cell_id, 1, 0)  # Create a binary mask for the current cell
            single_cell_mask = single_cell_mask.copy().astype(np.uint8)
            contours, _ = cv2.findContours(single_cell_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            crop_mask = np.zeros_like(single_cell_mask).astype(np.uint8)
            cv2.drawContours(crop_mask, contours, -1, 255, thickness=cv2.FILLED)

            w1 = images["w1"]
            w2 = images["w2"]
            w3 = images["w3"]
            w4 = images["w4"]
            w5 = images["w5"]
            w1_mask = np.copy(w1)
            w2_mask = np.copy(w2)
            w3_mask = np.copy(w3)
            w4_mask = np.copy(w4)
            w5_mask = np.copy(w5)

            # Set the rest of the image to 0 where the mask is 0
            w1_mask[crop_mask == 0] = 0
            w2_mask[crop_mask == 0] = 0
            w3_mask[crop_mask == 0] = 0
            w4_mask[crop_mask == 0] = 0
            w5_mask[crop_mask == 0] = 0

            indices = np.where(single_cell_mask != 0)
            if all(len(idx) == 0 for idx in indices):
                print(cell_id)
                continue
            y_min, x_min = np.min(indices, axis=1)
            y_max, x_max = np.max(indices, axis=1)

            # Calculate the center of the bounding box
            # center_z = (z_min + z_max) // 2
            center_y = (y_min + y_max) // 2
            center_x = (x_min + x_max) // 2

            # Calculate the coordinates of the top-left corner of the cropped box
            # box_zmin = center_z - box_size // 2
            box_ymin = center_y - box_size // 2
            box_xmin = center_x - box_size // 2
            if box_ymin < 0:
                box_ymin = 0
            if box_xmin < 0:
                box_xmin = 0

            # Crop the image and mask
            w1_crop = w1_mask[box_ymin : box_ymin + box_size, box_xmin : box_xmin + box_size]
            w2_crop = w2_mask[box_ymin : box_ymin + box_size, box_xmin : box_xmin + box_size]
            w3_crop = w3_mask[box_ymin : box_ymin + box_size, box_xmin : box_xmin + box_size]
            w4_crop = w4_mask[box_ymin : box_ymin + box_size, box_xmin : box_xmin + box_size]
            w5_crop = w5_mask[box_ymin : box_ymin + box_size, box_xmin : box_xmin + box_size]
            cropped_mask = single_cell_mask[box_ymin : box_ymin + box_size, box_xmin : box_xmin + box_size]
            composite_crop = np.stack([w1_crop, w2_crop, w3_crop, w4_crop, w5_crop], axis=-1)
            composite_crop_out = composite_crop.astype(np.uint8)
            
            if w1_crop.shape[0] < 100 or w1_crop.shape[1] < 100:
                #print(f"Skipping cell {cell_id} as it is out of frame.")
                continue

            #fig, axs = plt.subplots(1, 3, figsize=(12, 4))
            #axs[0].imshow(w2)
            #axs[0].set_title('Image 1')
            #axs[1].imshow(crop_mask)
            #axs[1].set_title('Image 2')
            #axs[2].imshow(w2_crop)
            #axs[2].set_title('Image 3')
            #plt.tight_layout()
            #plt.show()

            OutputDir = f'/share/data/analyses/benjamin/Single_cell_project/scDINO_RH30/RMS_masks/crops_new/'
            if not os.path.exists(OutputDir): 
                os.makedirs(OutputDir)
                
            file_string = image_path.split("/")[-1]
            first_part = file_string.split("_w")[0]

            #np.save(f'{OutputDir}/mask-tp-{tpstr}.npy', single_cell_mask)
            cv2.imwrite(f'{OutputDir}w1_crops/{first_part}_w1_{cell_id}.tif', w1_crop)
            cv2.imwrite(f'{OutputDir}w2_crops/{first_part}_w2_{cell_id}.tif', w2_crop)
            cv2.imwrite(f'{OutputDir}w3_crops/{first_part}_w3_{cell_id}.tif', w3_crop)
            cv2.imwrite(f'{OutputDir}w4_crops/{first_part}_w4_{cell_id}.tif', w4_crop)
            cv2.imwrite(f'{OutputDir}w5_crops/{first_part}_w5_{cell_id}.tif', w5_crop)
            tiff.imsave(f'{OutputDir}composite_crops/{first_part}_composite_{cell_id}.tif', composite_crop_out)
            #print("Saving cell", cell_id, "complete")

In [3]:
files = []
import glob, os
#os.chdir('/home/jovyan/share/mikro/RMS/RMS-e02-v01-P014561-RH30-48h/2021-10-29/826/')
folder_path = '/home/jovyan/share/mikro/RMS/RMS-e02-v01-P014561-RH30-48h/2021-10-29/826/'
files = os.listdir(folder_path)
# Filter the files that do not contain "thumb" in their filenames
filtered_files = [file for file in files if "thumb" not in file]
#for file in glob.glob("*w6.png"):
 #   if not file in files:
  #      files.append(file)
files_sorted = sorted(filtered_files)
image_paths = [os.path.join(folder_path, files_sort) for files_sort in files_sorted]

print(len(image_paths),str("images"))

5760 images


In [4]:
# Set the desired dimensions for cropping
crop_width = 100
crop_height = 100
box_size = 100
# Load your original image
original_image = img 

NameError: name 'img' is not defined

In [5]:
for key,value in tqdm(mask_dict.items(), total = len(mask_dict.items()), desc="Processing"):
    generate_single_cell_crops(value, key)

NameError: name 'mask_dict' is not defined